## My Spotify Wrapped Trends, since 2018

This is a notebook that performs some basic data processing operations on my own Spotify Wrapped playlists from 2018 (when I started using Spotify). 

*Note on Fetching Playlist Data:*

Fetching the data from Spotify's API is done using the `spotify` Python Library. Being an asynchronous implementation of the API, it didn't play too nice with Jupyter in my experience, so it has been separately implemented in `playlists.py`. By default, running the script returns `list.csv`, which is used as the data source in this notebook.


### Initialisation

Loading libraries and data.

In [ ]:
import pandas as pd
import numpy as np
import hashlib
import textwrap
import matplotlib.pyplot as plt

pd.set_option('mode.chained_assignment', None)

In [ ]:
# accessing csv (created by playlists.py script)

file = "list.csv"
df = pd.read_csv(file)
df.info()

### Data Processing

#### Creating 'song_id' column:

The CSV is all populated with datapoints directly fetched from the API. We must do some cleaning and other processing for our purposes.

1. Creating a 'combined' column, and then dropping it: There are some tracks in these playlists that are the exact same songs and performances, but have different Spotify IDs<sup>[1]</sup>. So we needed an id of our own, created by the concatenation of the song name and artist names. By making this field all lowercase, we also avoid another edge case<sup>[2]</sup> Not the most precise implementation<sup>[3]</sup>, but it works for our use case. 

2. The 'combined' column is then hashed to create 'song_id'. There is not much reason to hash the data; there can be any alternative method to make up an identifier of this data, but hashing in this case is more straightforward to create unique values from the 'combined' field (see [2])


<sup><sub>[1] Possibly being Single releases vs. Album tracks, etc.</sub></sup>

<sup><sub>[2] Some tracks can have some words capitalised or not ("the" vs. "The"), despite being the same tracks.</sub></sup>

<sup><sub>[3] Re-released music with changed song names (yes, this is the "(Taylor's Version)" edge case) would require you to match the titles in the DataFrame</sub></sup>

In [ ]:
# Combine "name" and "artists" columns into a new column "combined"
df['combined'] = (df['name'] + ' - ' + df['artists']).str.lower() # addressing the "the","The" issue

# Hash the combined values to create a unique identifier - some songs have different spotify IDs but are the exact same track
df['song_id'] = df['combined'].apply(lambda x: hashlib.md5(x.encode()).hexdigest())

df = df.drop(columns=['combined'])
# might need some further wrangling for tv tracks later - keeping older tracks as it is

#### Pivoting the DataFrame:

This is where song_id comes in handy. Songs appearing in more than one Wrapped lists have the same ID, so by pivoting over song_id as our index, for all the 'year' columns, we get a more manuverable table which we can use for all further processing. This creates a table with only song_id's and the yearly ranks. Songs that don't appear in some year's list get the value ``0`` for that year.

We add the song name (``name``) and artist name (``artists``) back in the next step, to bring together all distinct songs, despite them having different Spotify IDs.

In [ ]:
# pivoting table, for years on song_id

years = df['year'].unique().tolist() # final, don't redefine - useful later as well
song_info_short = ['name', 'artists']
pivot_df = df.pivot_table(index='song_id', columns='year', values='index', fill_value=0)

# Reset the index to make 'song_id' a regular column
pivot_df = pivot_df.reset_index()

# convert floats to int to make data cleaner
pivot_df[years] = pivot_df[years].astype(int)


In [ ]:
# adding back song names and artists on song_id values

if df['song_id'].duplicated().any():
    df = df.drop_duplicates(subset='song_id')
    
pivot_df = pd.merge(pivot_df, df[['name', 'artists', 'song_id']], on='song_id', how='left')
pivot_df # working dataset - data cleaned

#### Creating 'list_appearances' column: How many yearly lists is each song in?

The ``'list_appearances'`` column is added to view at-a-glance how many times has a track appeared in a yearly Wrapped list. It simply checks how many columns for each row have a non-zero value. 

In [ ]:
# counting number of appearances for each song in the lists
pivot_df['list_appearances'] = pivot_df[years].apply(lambda row: row.astype(bool).sum(), axis=1)

#### Finishing up Data Processing:

Just rearranging our data columns.

In [ ]:
# rearranging df to have all year indices to the end
cols = ['name', 'artists', 'song_id', 'list_appearances'] + years
pivot_df = pivot_df[cols]
pivot_df.head()

### Data Analysis

#### Appearances by Artists, and their multiple entries.

In [ ]:
# list of each unique artist and how many songs they have on the list
song_artists = pivot_df['artists'].value_counts()

# Keeping the ones with multiple entrants in a separate series
multi_song_artists = song_artists[song_artists > 1]

##### Creating a bar graph of all the artists with multiple entries

In [ ]:
# Creating a bar chart to visualise all multiple-entrant artists

grouped_multi_artists = multi_song_artists.groupby(multi_song_artists).apply(lambda x: ', '.join(x.index))

plt.figure(figsize=(12, 14))
plt.barh(grouped_multi_artists, grouped_multi_artists.index, color='lightgreen')

plt.xlabel('Number of Songs')
plt.ylabel('Artists')
plt.title('Track Counts by Artists')

plt.yticks(rotation=0, ha='right', wrap=True)
plt.xticks(range(0, max(grouped_multi_artists.index) + 1, 1))
plt.tight_layout()

for index, value in enumerate(grouped_multi_artists.index):
    plt.text(value + 0.25, index, str(value), va='center', ha='center', fontsize=10)

plt.show()

#### One-year Dream Runs

Ever had a very ephemeral earworm? A song that you listened to a whole lot, but only for a short while? Or, songs that were very "of a particular time" for you? They frequently end up in the Top 10 for me each year, and then completely vanish from my Wrapped the next year. Here's some data analysis on them! 

In [ ]:
# Creating a df mask to get each year's top 10

# For all years, create a mask for all rows with rank < 10 in any year
top_10s = pivot_df[pivot_df[years].apply(lambda x: x.between(1, 10)).any(axis=1)]

top_10s

In [ ]:
# Filtering all top 10s which vanished from the list the next year

dream_runs = pd.DataFrame()

def one_year_filter(row):
    for year in years[:-1]: # TODO: go by index in years, in case any year is skipped
        if ((row[year] < 10 and row[year] > 0) and row[year + 1] == 0): 
            return year
    return None

dream_runs = (top_10s.copy()
              .assign(dream_year=top_10s.apply(one_year_filter, axis=1))
              .dropna(subset=['dream_year'])
              .astype({'dream_year': int})
              .drop(columns=['song_id', 'list_appearances'])
              .sort_values(by='dream_year'))
dream_runs

##### Plot to visualise the Top 10 "dream run" tracks from each year recorded

In [ ]:
# scatter plot to visualise dream run tracks

scatter_dream_runs = dream_runs.copy()

scatter_dream_runs['pos'] = scatter_dream_runs.apply(lambda row: row[row['dream_year']], axis=1)
scatter_dream_runs['anno'] = scatter_dream_runs['artists'] + ' - ' + scatter_dream_runs['name']
scatter_dream_runs = scatter_dream_runs.drop(columns=years)

plt.figure(figsize=(8, 6))
plt.scatter(scatter_dream_runs['dream_year'], scatter_dream_runs['pos'], c='#00ed00')

for index, row in scatter_dream_runs.iterrows():
    wrap = textwrap.fill(row['anno'], width=20)
    plt.annotate(wrap, (row['dream_year'], row['pos']), textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)

plt.xlabel('Years')
plt.ylabel('Ranks')
plt.title('One-Year Dream Runs')

plt.yticks(range(1, 11, 1))
plt.xticks(range(min(scatter_dream_runs['dream_year']), max(scatter_dream_runs['dream_year']) + 1, 1))

plt.margins(x=0.2, y=0.1)
plt.grid(axis='x')

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().invert_yaxis()

plt.tight_layout()

plt.show()

#### On The Up

Tracks that ended up higher in a list than the previous year's list on one occasion, or returned to the lists after being absent for a year or more.

In [ ]:
# "recovering tracks - with >= 2 appearances, the ones that went up over a year in ranks, 
# or returned to the lists after being absent

two_df = pivot_df[pivot_df['list_appearances'] >= 2]
# two_df

# each row each year, if the next is larger than the previous, recovery, else not...?
recover_df = pd.DataFrame(columns=two_df.columns)
# print(years)

for index, row in two_df.iterrows():
    off_list = False
    for i in range(len(years) - 1):
        current_year = years[i]
        next_year = years[i + 1]

        curr_value = row[current_year]
        next_value = row[next_year]
        
        if curr_value != 0 and next_value == 0:
            off_list = True
        if (next_value < curr_value or off_list) and next_value != 0:
            recover_df = pd.concat([recover_df, row.to_frame().T], ignore_index=True)
            break

recover_df


In [ ]:
# viz. for recovering tracks and by how much did each go up (exclude off_list tracks)
# maybe expand on recover_df, add params ('off_list/climbed', 'upward delta (if climbed)')

#### First to Last Time

A closer look at your first Wrapped playlist. What tracks didn't return to the playlists ever again, which ones did, and which ones still found place in your latest Wrapped playlist!

In [ ]:
# How are the tracks from your first Wrapped doing in subsequent lists (if they made it in any lists afterwards,
# and the ones that didn't, where were they ranked?)

df_first = pivot_df[pivot_df[years[0]] > 0]
first_none_mask = df_first[years[1:]].eq(0).all(axis=1)

df_first_none = df_first[first_none_mask]
df_first_not_none = df_first[~first_none_mask]

print(f"Songs from {years[0]}'s Wrapped that were in subsequent lists = {df_first_not_none.shape[0]}")
print(f"Songs from {years[0]}'s Wrapped that were not in subsequent lists = {df_first_none.shape[0]}")

# TODO: (new) per-playlist attrition (needs more determination, what about previous lists)
# Songs present in the last Wrapped 


# df_first_not_none
# df_first.sort_values(by=years[0])

In [ ]:
# Songs only appearing in the following list

df_only_next = df_first_not_none[df_first_not_none[years[2:]].eq(0).all(axis=1)]

df_only_next['delta'] = df_only_next.apply(lambda row: row[years[1]] - row[years[0]], axis=1)
df_only_next[['name', 'artists', years[0], years[1], 'delta']].sort_values('delta')

In [ ]:
ls = list(range(1, 5, -1))
print(ls)

In [ ]:
# Songs from the first Wrapped present in the last Wrapped 

df_first_not_none[df_first_not_none[years[-1]] > 0]

## under development sections:

In [ ]:
# DEV

top_10s[(top_10s[2019] > 0) & (top_10s[2019] <= 10)].sort_values(by=2019)

In [ ]:
top_10s[top_10s[years].eq(3).any(axis=1)]

# df_first_not_none[df_first_not_none[years[2:]].eq(0).all(axis=1)]

In [ ]:
year_to_refer = 2022
artist_to_refer = 'Wolfgang Amadeus Mozart'
pivot_df[(pivot_df['artists'] == artist_to_refer) & (pivot_df[year_to_refer] > 0)][['name', 'artists', year_to_refer]]

In [ ]:
pivot_df[(pivot_df['artists'] == "Charli XCX")] #[['name', 'artists']]

In [ ]:
pivot_df[pivot_df['name'] == 'pink diamond']

In [ ]:
# present in 3 or more lists

three_df = pivot_df[pivot_df['list_appearances'] >= 3]

three_df

In [ ]:
# present in 2 or more lists

two_df = pivot_df[pivot_df['list_appearances'] >= 2]

two_df

In [ ]:
# present in 4 or more lists

four_df = pivot_df[pivot_df['list_appearances'] >= 4]

four_df

In [ ]:
# one timers

one_df = pivot_df[pivot_df['list_appearances'] == 1]

one_df

In [ ]:
# generic line graph maker - for reference

sample_df = three_df
years = sample_df.columns[4:]

for index, row in sample_df.iterrows():
    non_zero_values = [(year, value) for year, value in zip(years, row[4:]) if value != 0]
    if non_zero_values:
        years_non_zero, values_non_zero = zip(*non_zero_values)
        plt.plot(years_non_zero, values_non_zero, marker='D', label=row['name'])
        
# Adding title and labels
# plt.figure(figsize=(15, 15))

plt.title('Line Graph for Songs Over Years')
plt.xlabel('Year')
plt.ylabel('Rank in Wrapped Playlist')

plt.xticks(range(int(min(years)), int(max(years)) + 1))

# Adding a legend to identify each line
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.gca().invert_yaxis()

# Display the graph
plt.show()

In [ ]:
# some sort of statistical metric to track consistency of the track in the rankings where it's present in >3

In [ ]:
# one list to another: which adjacent have the greatest overlap?

In [ ]:
# (merge with recovering tracks) comebacks (in list -> 0 -> back in list)

In [ ]:
# artists' best years, most common apperances, ups-and-downs over the years 
# an illustration: MGMT - 2021 super dense, fell off after

In [ ]:
# 3 or more appearances, graphing them